In [1]:
from ultralytics import YOLO
import os


In [2]:
# Dataset directory
DATASET_DIR = "/home/sandeshprasai/Final_Semester_Project/AI_Attendance_System/ai-ml-model/DataSets/YoloV8/wider_yolo"

# YOLOv8 dataset YAML
DATA_YAML = "/home/sandeshprasai/Final_Semester_Project/AI_Attendance_System/ai-ml-model/DataSets/YoloV8/wider_yolo/datasets.yaml"


# Model save directory
MODEL_SAVE_DIR = "/home/sandeshprasai/Final_Semester_Project/AI_Attendance_System/ai-ml-model/src/models"
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

# Device (0 = GPU, 'cpu' for CPU)
DEVICE = 0


In [3]:
# Hyperparameters for small faces detection
EPOCHS = 100
BATCH_SIZE = 16
IMG_SIZE = 512  # can adjust to 512 for faster training
LR0 = 1e-3       # initial learning rate
PATIENCE = 20    # early stopping patience


In [4]:
# Use pretrained YOLOv8 nano model
model = YOLO("yolov8n.pt")  # you can also try yolov8s.pt for bigger backbone


In [ ]:
model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    optimizer="AdamW",
    lr0=LR0,
    patience=PATIENCE,
    device=DEVICE,
    project=MODEL_SAVE_DIR,
    name="yolov8_wider_face",  # subfolder for this training run
    exist_ok=True,             # overwrite if folder exists
    augment=False,             # disable heavy augmentations
    mosaic=0,                  # disable mosaic
    mixup=0,                   # disable mixup
    copy_paste=0,              # disable copy-paste
    freeze=10,                 # freeze backbone for first 10 epochs
    rect=True,                 # rectangular training to save memory
    workers=4,                 # reduce dataloader workers
    half=True,                  # FP16 training to reduce GPU memory usage
    plots=False  # <-- DISABLE THIS
)


In [6]:
# Path to the best saved weights
best_model_path = os.path.join(MODEL_SAVE_DIR, "yolov8_wider_face", "weights", "best.pt")
print("Best model saved at:", best_model_path)


Best model saved at: /home/sandeshprasai/Final_Semester_Project/AI_Attendance_System/ai-ml-model/src/models/yolov8_wider_face/weights/best.pt


In [4]:
import cv2
from ultralytics import YOLO
import torch

# -----------------------------
# Configuration
# -----------------------------
MODEL_PATH = "../../src/models/0_yolov8_wider_face_DetectionModel/weights/best.pt"
CONF_THRESHOLD = 0.4
IMG_SIZE = 640
DEVICE = 0 if torch.cuda.is_available() else "cpu"

# -----------------------------
# Load YOLOv8 Face Detector
# -----------------------------
model = YOLO(MODEL_PATH)
model.to(DEVICE)

# -----------------------------
# Open Camera
# -----------------------------
# Use 0 for laptop webcam
# Or IP camera URL if needed
cap = cv2.VideoCapture("http://192.168.1.81:4747/video")

if not cap.isOpened():
    raise RuntimeError("❌ Camera not accessible")

print("✅ Camera started. Press 'q' to exit.")

# -----------------------------
# Inference Loop
# -----------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO inference
    results = model(
        frame,
        imgsz=IMG_SIZE,
        conf=CONF_THRESHOLD,
        device=DEVICE,
        verbose=False
    )

    # Draw detections
    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(
                frame,
                f"Face {conf:.2f}",
                (x1, y1 - 8),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 255, 0),
                1
            )

    cv2.imshow("YOLOv8 Face Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# -----------------------------
# Cleanup
# -----------------------------
cap.release()
cv2.destroyAllWindows()


✅ Camera started. Press 'q' to exit.
